In [1]:
import numpy as np
import pandas as pd 
import os
import random
import time

In [7]:
DEBUG_MODEL=True
INPUT_DIR="/vc_data/users/v-mxiong/qc/origin/LabelData"
SCHEMA_DIR="/vc_data/users/v-mxiong/qc/origin/Schema"
SEED=42
def read_data_txt(input_file,schema_file,is_debug=DEBUG_MODEL):
    with open(schema_file,"r") as input_object:
        schema_line=input_object.readlines()
    head=schema_line[0].strip().split("\t")
    with open(input_file,"r") as input_object:
        input_lines=input_object.readlines()
    if DEBUG_MODEL==True:
        input_lines=input_lines[:50000]
    input_lines=[ line.strip().split("\t") for line in input_lines]
    input_lines=[ line for line in input_lines if len(line)==len(head)]
    data_rows=input_lines
    input_df=pd.DataFrame(data_rows,columns=head)
    return input_df
def sample_negative(df,negative_rate=1):
    negative_df=df[df.label==0]
    positive_df=df[df.label==1]
    sample_num=len(positive_df)*negative_rate
    
    sample_df=negative_df.sample(n=sample_num,random_state=SEED)
    
    # .sample(frac=1) 用于打乱顺序
    return_df=pd.concat([positive_df,sample_df]).sample(frac=1)
    return_df=return_df.reset_index(drop=True)
    return return_df

In [8]:
input_df=read_data_txt(INPUT_DIR,SCHEMA_DIR)
input_df["label"]=input_df["label"].replace(["0","1"],[0,1])

In [10]:
import langid
from tqdm._tqdm_notebook import tqdm_notebook
from tqdm import tqdm
# import fasttext
def language_classify(df):

    tqdm_notebook.pandas(desc='apply')
#     df["language"]=df.progress_apply(lambda r: langid.classify(r["Query"])[0], axis=1)
    df["language"]=df.Query.progress_apply(lambda r: langid.classify(r)[0])
    df_en=df[df["language"]=="en"]
    print("filter rate:",len(df_en)/len(df))
    return df,df_en
# def language_fasttext_classify(df):
#     path_to_pretrained_model = '/vc_data/users/v-mxiong/model_file/lid.176.bin'
#     fmodel = fasttext.load_model(path_to_pretrained_model)
#     tqdm_notebook.pandas(desc='apply')
#     df["language"]=df.Query.progress_apply(lambda r: fmodel.predict(r)[0][0][9:])
#     df_en=df[df["language"]=="en"]
    
#     print("filter rate:",len(df_en)/len(df))
#     return df,df_en
# input_lang_df=language_classify(input_df)

def select_market(df,language="en"):

    tqdm_notebook.pandas(desc='apply')
    return_choose=input_df.Market.progress_apply(lambda r:r[0:2]==language)
    
    return_df=df[return_choose]
    print("filter rate:",len(return_df)/len(df))
    
    return return_df

In [11]:
input_df["label"].value_counts()
sat_rate=input_df["label"].mean()
input_df["ClickCount"]=input_df["ClickCount"].astype(int)
click_rate=(input_df["ClickCount"]>0).mean()
noclick_rate=1-click_rate
show_df=pd.DataFrame({"query_num":[len(input_df)],"click_rate":[click_rate],\
"noclick_rate":[noclick_rate],"sat_rate":[sat_rate],"dsat_rate":[click_rate-sat_rate]})
show_df

,query_num,click_rate,noclick_rate,sat_rate,dsat_rate
0,49870,0.030339,0.969661,0.020553,0.009785


In [12]:
sum_click=sum(input_df["ClickCount"])
sum_count=sum(input_df["Count"].astype(int))
sum_click/sum_count

0.005249904080496728

In [13]:
# _,input_df=language_classify(input_df)
# input_df_lang,input_df=language_fasttext_classify(input_df)
input_df=select_market(input_df)


filter rate: 0.6095849207940646


In [14]:
sample_df=input_df.sample(n=1000)

In [15]:
# with open(data_file,"w") as file_object:
#     for i in tqdm(range(sample_df.shape[0])):
#         s="\t".join(sample_df.iloc[i].astype(str))
        
#         file_object.writelines(s)

# data_file="/vc_data/users/v-mxiong/qc/labeldata_lang"
# schema_file="/vc_data/users/v-mxiong/qc/schema_lang"
# input_df_lang.to_csv(data_file,header=0,index=0,sep="\t")
# with open(schema_file,"w") as file_object:
#     file_object.writelines("\t".join(input_df_lang.columns.tolist()))

In [16]:
sample_df=sample_negative(input_df)

In [21]:
sample_df[sample_df["label"]==1].sample(n=50)[["Query","label"]]

,Query,label
53,air america movie,1
761,adobe photoshop elements 2019 download,1
972,automtic save,1
341,about:blankalexander siggid,1
518,auditory processing goals speech therapy goals...,1
388,arigato,1
129,amazon address headquarters,1
1171,achari curry,1
1096,1997 illinois accessibility code pdf,1
575,atwood realty edisto beach rentals,1


In [22]:
sample_df[sample_df["label"]==0].sample(n=100)[["Query","label"]]

,Query,label
1176,agrowilg,0
730,1964 1/2 mustang for sale,0
758,18-4200-93,0
565,aricom ? was a mining and metals company from ...,0
274,20 mmammo cost,0
115,______ co + ______ h2 → _______ ____________,0
210,baby boy names that go with hispanic last names,0
1229,2517 indian sage way,0
912,"15905 sw loon dr beaverton, or",0
329,acrynom rr,0
